# SDG Chapter 24 - Advanced Analytics and Machine Learning

Converted from the py file in the repo (using p2j) and then manualy fixed.


In [ ]:
from pyspark.sql import SparkSession
#import pyspark.sql.functions as f
#from pyspark.sql.types import *
spark = SparkSession.builder.appName('MLlib').getOrCreate()
datapath = "../../data/sdg/"

Vectors can be dense or sparse. Both can be used by the MLlib

In [ ]:
from pyspark.ml.linalg import Vectors
denseVec = Vectors.dense(1.0, 2.0, 3.0)
size = 3
idx = [1, 2] # locations of non-zero elements in vector
values = [2.0, 3.0]
sparseVec = Vectors.sparse(size, idx, values)

In [ ]:
df = spark.read.json(datapath + "/simple-ml")
df.orderBy("value2").toPandas()

## Feature engineering with Transformers
The RFormula is a method to select fields. It borrows from R language.

The same processing can be done using other API calls.

In [ ]:
from pyspark.ml.feature import RFormula
supervised = RFormula(formula="lab ~ . + color:value1 + color:value2")

In [ ]:
fittedRF = supervised.fit(df)   # apply the formula to the df
preparedDF = fittedRF.transform(df) # ...and actually perform the addition of 'features'
preparedDF.toPandas()

Now we have preprocessed data.

Let's train a model on the data

In [ ]:
train, test = preparedDF.randomSplit([0.7, 0.3])

In [ ]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(labelCol="label",featuresCol="features") # Use the default parameters.

In [ ]:
# list of all the LogisticRegression class hyper parameters.
print(lr.explainParams())

Train the model. This transformation is 'eager', done immediately.

In [ ]:
fittedLR = lr.fit(train)

We can now use the mode to make predictions:

In [ ]:
fittedLR.transform(train).select("label", "prediction").show(3)

## Model evaluation

### Estimators
The MLlib has several classes to estimate model performance.

We now **repeat** the code above, this time performing grid search on the hyper params.

In [ ]:
train, test = df.randomSplit([0.7, 0.3])

In [ ]:
rForm = RFormula()
lr = LogisticRegression().setLabelCol("label").setFeaturesCol("features")

Instead of manually
using our transformations and then tuning our model we just make them stages in the overall
pipeline

In [ ]:
from pyspark.ml import Pipeline
stages = [rForm, lr]
pipeline = Pipeline().setStages(stages)

Now that you arranged the logical pipeline, the next step is training. In our case, we won’t train
just one model (like we did previously); we will train several variations of the model by
specifying different combinations of hyperparameters that we would like Spark to test. We will
then select the best model using an Evaluator that compares their predictions on our validation
data.

In [ ]:
from pyspark.ml.tuning import ParamGridBuilder
params = ParamGridBuilder()\
  .addGrid(rForm.formula, [
    "lab ~ . + color:value1",
    "lab ~ . + color:value1 + color:value2"])\
  .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])\
  .addGrid(lr.regParam, [0.1, 2.0])\
  .build()

We built a grid with 2 Rformula, 3 ElasticNet, 2 Regularization params.

Overall 2 * 3 * 2 = 12 combinations.


Now that the grid is built, it’s time to specify our evaluation process. The evaluator allows us to
automatically and objectively compare multiple models to the same evaluation metric. There are
evaluators for classification and regression, covered in later chapters, but in this case we will use
the BinaryClassificationEvaluator

The possible metrics are areaUnderPR and areaUnderROC

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator()\
  .setMetricName("areaUnderROC")\
  .setRawPredictionCol("prediction")\
  .setLabelCol("label")

Now that we have a pipeline that specifies how our data should be transformed, we will perform
model selection to try out different hyperparameters in our logistic regression model and
measure success by comparing their performance using the areaUnderROC metric.

We could also use CrossValidator instead of TrainValidationSplit.

Remember that we MUST NOT use the test data!


In [ ]:
from pyspark.ml.tuning import TrainValidationSplit
tvs = TrainValidationSplit()\
  .setTrainRatio(0.75)\
  .setEstimatorParamMaps(params)\
  .setEstimator(pipeline)\
  .setEvaluator(evaluator)

In [ ]:
tvsFitted = tvs.fit(train)

How did it perform on the test set?

In [ ]:
evaluator.evaluate(tvsFitted.transform(test))

Let's see a summary for some of the (12) models we tries

In [ ]:
 trainedPipeline = tvsFitted.bestModel

In [ ]:
 TrainedLR = trainedPipeline.stages[1] # stages[0] has the RFormulamodel

In [ ]:
# See the loss (== objective function) in each iteration
# We can see that the loss converges to 0.43 . Is it good enough?
TrainedLR.summary.objectiveHistory

In [ ]:
# see what we can get from the summary (or just look at the doc)
[ x for x in dir(TrainedLR.summary) if x[0] != '_']

In [ ]:
TrainedLR.summary.precisionByLabel

In [ ]:
TrainedLR.summary.recallByThreshold.toPandas()

In [ ]:
TrainedLR.getRegParam()


We can save the model, to use it later for predictions:

In [ ]:
tvsFitted.write().overwrite().save('lr_model_very_nice')